In [ ]:
import os
import requests 
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('DEEPSEEK_API_KEY') 
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

In [ ]:
ds_system_message = "你是一个台湾人，你有严重的台湾腔，跟用户聊天时，要多用“干！”、“鸡掰、“甘霖娘鸡掰”、“醒磨”（就是什么的意思）、“超扯的！”、“没在怕哦”、“靠北”、“靠妖”等台湾常用语，并在每句话结尾要么加上“哦~”或者加上“了啦”，所有输出都用中文繁体字”"

In [ ]:
def chat(message, history):
    relevant_system_message = ds_system_message   # Python 不允许函数内无声明地修改全局变量，所以引入新变量relevant_system_message
    relevant_system_message += """每次回答不需要太多无关紧要的答案，比如你的回答“幹！今晚哦~ 靠北，我們可以去夜市吃小吃啦~ 甘霖娘雞掰，夜市超多好吃的啦！ 沒在怕哦，我們還可以去看電影哦~ 你覺得怎麼樣了啦？”不需要“ 沒在怕哦，我們還可以去看電影哦~ 你覺得怎麼樣了啦？”前面的回答就已经很够了，不需要在结尾还要加上“没在怕哦”和任何反问我的想法的事情。"""
    # messages 是一个列表，先加入系统消息（ds_system_message），然后加上历史消息（history），最后加入当前用户消息（message）
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = client.chat.completions.create(
        model='deepseek-chat', 
        messages=messages, 
        stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(
    fn=chat, 
    type="messages").launch()